### Simulation code ###
Date: June 04, 2025

Author: Ruben A. Proano

To use the code, please request permission by emailing rpmeie@rit.edu

This code is used to support a simulation study to analyze the impact of African vaccine manufacturing on the GAVI vaccine market.

This code is an implementation of the simulation discussed in the associated paper. 

This code is licensed under a **Creative Commons Attribution 4.0 International License** https://creativecommons.org/licenses/by/4.0/

Attribution the data and code should include:

    Ruben A. Proano, Rochester Institute of Technology, rpmeie@rit.edu
    Copyright notice (© 2025 R.Proano)
    Add a link to the original work (paper link)


### Content:###

- 1: General notation (Assumptions and Factors)
- 2: Function Definitions
- 3: Experimental design - notation description
- 4: Main Simulation Function
- 5: Input data upload
- 6: Simulation execution 
- 7: Output collection

## 1. General notation

### Assumptions

- Market 1== domestic market == gavi market \ AU
- Market 2== overseas market == AU market
- If AU produces a vaccine to satisfy overseas market, its production capacity is bounded by the AU's antigen demand
- Model based for 1 vaccine at the time
- DG = current GAVI market demand
- D1 = demand of non-AU GAVI countries. D1~N(u(D1), v(D1)) around the current demand levels of non-AU Gavi countries
- D2 = demand of AU countries. D2~N(u(D2), v(D2)) around the current demand levels for AU countries supported by non-AU manufacturers
- KG = current domestic capacity prior AU market
- K1 = domestic (non-AU) production capacity 
- K2 = overseas (AU) capacity
- KG = K1 + K2

- v1 = profit per dose of vaccine made in domestic market and sold in domestic market (non-AU GAVI countries)
- v2 = profit per dose of vaccines made in overseas market and sold in overseas market (AU Gavi countries)
- v3 = profit per dose of vaccines made in domestic market but sold overseas (made in non-AU Gavi countries and sold in AU Gavi countries)
- vi = pi- mi, where pi is selling price, mi production cost, which includes variable and fixed cost/unit (i=1,2,3)

### Factors
- Capacities (K1, K2). Key factor K2, given that K1 = KG - K2, and KG assumed constant
- Profit per unit vector (vD, vO, vT)
- let profit 0 <= vi <= (1+b)p, where p is the current UNICEF price per dose
- then vi~ U[0.1p, (1+b)p], b= 0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0


## 2. Function definitions

In [1]:
def ProductionPlan(profit:tuple, capacity:tuple, demand:tuple)-> tuple:
    scenario=PricingScenario(profit)
    plan = Plan(scenario, capacity, demand)
    return plan

In [2]:
import numpy as np

In [3]:
def ProfitFromPlan(plan:tuple, profit:tuple)->float:
    TotProfit= np.array(plan)*np.array(profit)
    return TotProfit

In [4]:
def PricingScenario(pft: tuple)-> int:
#
#Receives a tuple of three profits per dose (0: domestic, 1: overseas market, 2: transhipment)
#- checks if the tuple has three elements, 
#- checks if any tuple entries are equal
#- determines pricing scenario among markets

    try:
        if Checktuple(pft) ==0:
            raise ValueError("Either the input tuple has wrong number of entries, or has at least one equal entry")
        else:
            if pft[0]>pft[1]>pft[2] :
                return 1 #DO(4)
            elif pft[0]>pft[2]>pft[1]:
                return 2 #DT (3)
            elif pft[1]>pft[0]>pft[2]:
                return 3 #OD (2)
            elif pft[1]>pft[2]>pft[0]:
                return 4 #OT (1)
            elif (pft[2]>max(pft[0],pft[1]) and pft[2]< (pft[0]+pft[1])):
                return 5 #TM (5)
            else:
                return 6 #TE (6)
    except ValueError as error:
        return str(error)
    

In [5]:
def Checktuple(pft: tuple) ->int:
    if len(pft) != 3:
        return 0
    
    if pft[0]==pft[1] or pft[0]==pft[2] or pft[1]==pft[2]:
        return 0
    else:
        return 1
    

In [6]:
def Plan(scenario:int, capacity:tuple, demand:tuple)->tuple:
    if scenario==1:
        #print("scenario 1, DO")
        return DO(capacity, demand)
    elif scenario==2:
        #print("scenario 2, DT")
        return DT(capacity,demand)
    elif scenario==3:
        #print("scenario 3, OD")
        return OD(capacity, demand)
    elif scenario==4:
        #print("scenario 4, OT")
        return OT(capacity, demand)
    elif scenario==5:
        #print ("scenario 5, TM")
        return TM(capacity, demand)
    else:
        #print ("scenario 6, TE")
        return TE(capacity, demand)
        

In [7]:
def DO(capacity:tuple, demand:tuple)-> tuple:
    if (demand[0]<=capacity[0] and demand[1]<=capacity[1]):
      # print('area 0')
        return (1, 'a_0', demand[0], demand[1], 0)
    elif (demand[0]<=capacity[0] and demand[1]>capacity[1] and (demand[0]+demand[1]<=capacity[0]+capacity[1])):
      # print('area 1')
        return (1, 'a_1', demand[0], capacity[1], demand[1]-capacity[1])
    elif (demand[0]<=capacity[0] and demand[1]>capacity[1] and (demand[0]+demand[1]>capacity[0]+capacity[1])):
      # print('area 2')
        return (1, 'a_2', demand[0], capacity[1], capacity[0]-demand[0])
    elif (demand[0]>capacity[0] and demand[1]>capacity[1]):
      # print('area 3')
        return (1, 'a_3', capacity[0], capacity[1], 0)
    else:
      # print('area 4')
        return (1, 'a_4',capacity[0], demand[1],0)

In [8]:
def DT(capacity:tuple, demand:tuple)->tuple:
    if(demand[0]<=capacity[0] and (demand[0]+demand[1]<=capacity[0])):
        #print('area 0')
        return(2,'a_0', demand[0], 0, demand[1])
    elif (demand[0]<=capacity[0] and (capacity[0]<demand[0]+demand[1]<=capacity[0]+capacity[1])):
        #print('area 1')
        return (2,'a_1', demand[0], demand[0]+demand[1]-capacity[0], capacity[0]-demand[0])
    elif (demand[0]<=capacity[0] and (demand[0]+demand[1]> capacity[0]+capacity[1])):
        #print('area 2')
        return (2, 'a_2', demand[0], capacity[1], capacity[0]-demand[0])
    elif (demand[0]>capacity[0] and demand[1]>capacity[1]):
        #print('area 3')
        return (2, 'a_3', capacity[0], capacity[1], 0)
    else:
        #print('area 4')
        return(2, 'a_4', capacity[0], demand[1], 0)

In [9]:
def OD(capacity:tuple, demand:tuple)-> tuple:
    if(demand[0]<=capacity[0] and demand[1]<=capacity[1]):
        #print('area 0')
        return(3, 'a_0', demand[0], demand[1], 0)
    elif (demand[0]<=capacity[0] and demand[1]> capacity[1] and (demand[0] + demand[1]<= capacity[0]+capacity[1])):
        #print('area 1')
        return (3, 'a_1', demand[0], capacity[1], demand[1]-capacity[1])
    elif (demand[0]<=capacity[0] and (demand[0] + demand[1]> capacity[0]+capacity[1])):
        #print('area 2')
        return (3, 'a_2', demand[0], capacity[1], capacity[0]-demand[0])
    elif (demand[0]>capacity[0] and demand[1]>capacity[1]):
        #print('area 3')
        return (3, 'a_3', capacity[0], capacity[1], 0)
    else:
        #print('area 4')
        return (3, 'a_4', capacity[0], demand[1], 0)
    

In [10]:
def OT(capacity:tuple, demand:tuple)->tuple:
    if(demand[0]<=capacity[0] and demand[1]<=capacity[1]):
        #print('area 0')
        return(4,'a_0', demand[0], demand[1], 0)
    elif (demand[0]<=capacity[0] and demand[1]> capacity[1] and (demand[0] + demand[1]<= capacity[0]+capacity[1])):
        #print('area 1')
        return (4,'a_1', demand[0], capacity[1], demand[1]-capacity[1])
    elif (capacity[1]<demand[1]<=capacity[0]+capacity[1] and (demand[0]+demand[1]>capacity[0]+capacity[1])):
        #print('area 2')
        return (4,'a_2',capacity[0]+capacity[1]-demand[1], capacity[1], demand[1]-capacity[1])
    elif (demand[1]> capacity[0]+capacity[1]):
        #print('area 3')
        return (4, 'a_3', 0, capacity[1], capacity[0])
    else:
        #print('area 4')
        return(4, 'a_4',capacity[0], demand[1], 0)

In [11]:
def TM(capacity:tuple, demand:tuple)->tuple:
    if(demand[0]<=capacity[0] and (demand[0]+demand[1]<=capacity[0])):
        #print('area 0')
        return (5, 'a_0',demand[0], 0, demand[1])
    elif (demand[0]<=capacity[0] and (capacity[0]< demand[0]+demand[1]<=capacity[0]+capacity[1])):
        #print('area 1')
        return (5, 'a_1', demand[0], demand[0]+demand[1]-capacity[0], capacity[0]-demand[0])
    elif ((capacity[1]<demand[1]<= capacity[0]+ capacity[1]) and (demand[0]+demand[1]>capacity[0]+capacity[1])):
        #print('area 2')
        return(5, 'a_2', capacity[0]+capacity[1]-demand[1], capacity[1], demand[1]-capacity[1])
    elif (demand[1]> capacity[0]+capacity[1]):
        #print('area 3')
        return (5, 'a_3', 0, capacity[1], capacity[0])
    else:
        #print('area 4')
        return (5, 'a_4', capacity[0], demand[1], 0)

In [12]:
def TE(capacity:tuple, demand:tuple)->tuple:
    if (demand[0]<=capacity[0] and demand[1]<= capacity[0] and demand[0]+demand[1]<= capacity[0]):
        #print('area 0')
        return (6, 'a_0', demand[0], 0, demand[1])
    elif(demand[1]<=capacity[0] and demand[0]+demand[1]>capacity[0]):
        #print('area 1')
        return (6, 'a_1', capacity[0]-demand[1], 0, demand[1])
    elif(capacity[0]<demand[1]<=capacity[0]+capacity[1]):
        #print('area 2')
        return (6, 'a_2', 0, demand[1]-capacity[0], capacity[0])
    else:
        #print('area 3')
        return (6, 'a_3',0, capacity[1], capacity[0])

In [13]:
def create_tuple():
    # Import random module to generate random numbers
    import random
    
    # Generate three random numbers between 0 and 100
    num1 = random.randint(0, 100)
    num2 = random.randint(0, 100)
    num3 = random.randint(0, 100)
    
    # Ensure all three numbers are different
    while num2 == num1:
        num2 = random.randint(0, 100)
    while num3 == num1 or num3 == num2:
        num3 = random.randint(0, 100)
    
    # Create and return tuple of the three numbers
    return (num1, num2, num3)

In [14]:
def Benchmark_Orig(capacity:tuple, demand:tuple)->tuple:
    if(demand[0]+demand[1]<=capacity[0]):
        return(demand[0], 0, demand[1])
    elif(demand[0]<=capacity[0] and (demand[0]+demand[1]>capacity[0])):
        return (demand[0], 0, capacity[0]-demand[0])
    else:
        return (capacity[0],0, 0)

In [15]:
def Benchmark(profit:tuple,capacity:tuple, demand:tuple)->tuple:
    """ assumes that there is no capacity in overseas market [1]. All production capacity is in domestic market [0]"""
    capacityL=list(capacity)
    capacityL[0]=capacity[0]+ capacity[1]
    capacityL[1]=0

    capacity=tuple(capacityL)
    plan=ProductionPlan(profit, capacity, demand)
    return(plan[-3:])
    

In [16]:
def Experiment(capacities,g, b, LD1, UD1, LD2, UD2 , rep ,pt:tuple):
    
    zv1= sc.truncnorm.rvs((pt[2]-pt[0])/pt[1],(pt[3]-pt[0])/pt[1])
    zv2= sc.truncnorm.rvs((pt[2]-pt[0])/pt[1],(pt[3]-pt[0])/pt[1])
    zv3= sc.truncnorm.rvs((pt[2]-pt[0])/pt[1],(pt[3]-pt[0])/pt[1])

    
    v1= zv1*pt[1]+pt[0]
    v2= zv2*pt[1]+pt[0]
    v3= zv3*pt[1]+pt[0]
    
    profits = (v1, v2, v3)
    relprof=(v2/v1,v3/v1) # relative profits of v0 and vT wrt vD.
    demands= (sc.uniform.rvs(LD1,UD1), sc.uniform.rvs(LD2,UD2))
    plan=ProductionPlan(profit=profits, capacity=capacities, demand=demands)
    bmkplan=Benchmark(profit=profits,capacity=capacities, demand=demands)
    Prof=np.array(plan[2:])*np.array(profits)
    bmkprof=np.array(bmkplan)*np.array(profits)
    yield (g,rep),relprof+capacities+plan+tuple(Prof)+tuple(bmkprof)
     

## 3. Experimental design - notation description

### Factors:
- production capacities K2 as a factor of KG
- K2 = g KG, where g=0.1, 0.2, 0.3, 0.4, 0.5, 0.6 , 0.7, 0.8, 0.9, 1.0.
- K1 = (KG-K2)
- Vector of random profits per unit vector (vD, vO, vT)
- let vi<= p, where p is the current UNICEF price per dose
- let vi~ U[0.1 p, (1+b)p], where b~U(0,1)
- explore b= 0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0

### Simulation:
- for each g level x for each b level
randomly generate 10,000 profit vectors
- number of experiments : |g-levels| x |b-levels| x 10,000 = 10x10x10,000 = 1,000,000 exp
for each (K1, K2) and profit tuple:
    - evaluate Production plan, Revenue from plan


## Experimentation:

main experimentation function:
receives a dictionary with the following information for each vaccine:

KG: expected capacity for current Gavi Market

ED1: expected demand in non-AU Gavi Countries

LD1: lower limit of demand in non-AU Gavi Countries

UD1: upper limit of demand in non-AU Gavi Countries

ED2: expected vaccine demand in AU Gavi Countries

LD2: lower limit of demand in AU Gavi Countries

UD2: upper limit of demand in AU Gavi Countries

(p, pmin, pmax, sp) : tuple with avg vaccine price, lower limit of vaccine price per dose, upper limit of vaccine price per dose, standard deviation of price per dose

results: {xD, xO, xT, vD, vO, vT, pfD, pfO, pfT, bpfD,bpfO, bpfT}

(xD, xO, xT): fullfilment quantities in Domestic, Overseas, Transhipment - if supply is avaiable from nonAU and AU Gavi countries

(pfD, pfO, pfT): profits in Domestic, Overseas, Transhipment - if supply is avaiable from nonAU and AU Gavi countries

(bpfD, bpfO, bpfT): benchmark profits - if supply is avaiable only from nonAU Gavi countries


### 4. Main Simulation function

In [17]:
import numpy as np
import scipy.stats as sc
import pandas as pd

In [18]:
def mainf(vac:dict):

    maindf=pd.DataFrame()
    
    for i in vac:
            
        KG = vac[i]['KG']   # expected capacity for current Gavi market
        ED1 = vac[i]['ED1'] # expected demand in non-AU Gavi countries
        LD1 = vac[i]['LD1'] # lower limit of demand in non-AU Gavi countries
        UD1 = vac[i]['UD1'] # upper limit of demand in non-AU Gavi countries
        ED2 = vac[i]['ED2'] # expected demand in AU Gavi countries
        LD2 = vac[i]['LD2'] # lower limit of demand in AU Gavi countries
        UD2 = vac[i]['UD2'] # upper limit of demand in AU Gavi countries

        b = 0.15
        p = vac[i]['p'] # vaccine price
        pmin= vac[i]['pmin'] # min price
        pmax=vac[i]['pmax'] # max price
        sp=vac[i]['sp'] # sqrt of average price variance
        numtest = 10000
    
        results={}
        for g in np.arange(0,1.2,0.2).round(1).tolist():
            K2 = g*ED2
            K1= KG-K2
            capacities =(K1, K2)
            experimentiter=(Experiment(capacities,g, b, LD1, UD1, LD2, UD2 , rep ,pt=( p, sp, pmin,pmax)) for rep in range(numtest))
            for val in experimentiter:
                results.update(val)
        
        df= pd.DataFrame.from_dict(results, orient='index').reset_index()   
        df.columns=['g_rep', 'v2/v1','v3/v1','K1','K2','scenario', 'area', 'xD', 'xO', 'xT', 'pfD', 'pfO', 'pfT', 'bpfD','bpfO','bpfT' ]    
        df['pDtot']=df['pfD']+df['pfT']
        df['bpfDtot']=df['bpfD']+df['bpfT']
        aa=df['g_rep'].apply(pd.Series)
        aa.columns=['g','rep']
        df['g']=aa['g']
        df['rep']=aa['rep']
        df['vac']=i
        
        maindf=pd.concat([maindf,df],  ignore_index=True)
        #print(df)
        print('done with vac', i)
    first_column=maindf.pop('vac')
    maindf.insert(0,'vac',first_column)
    print ("Finally done!")  
    return maindf

## 5. Input data upload

In [19]:
HPV={'KG': 5806000, 'ED1':3135240, 'LD1':1620000, 'UD1':6496200, 'ED2':2670760,'LD2':1380000, 
     'UD2':5533800, 'p':4.5, 'pmin': 4.45, 'pmax': 4.55, 'sp':0.001}

IPV={'KG': 65008000, 'ED1':24703040, 'LD1':19380000, 'UD1':34975200, 'ED2':40304960,'LD2':31620000, 
     'UD2':57064800, 'p':1.694, 'pmin': 0.83, 'pmax': 3.3, 'sp':0.9885}

Measles={'KG': 149076000, 'ED1':0, 'LD1':0, 'UD1':0, 'ED2':149076000,'LD2':117000000, 
     'UD2':170000000, 'p':0.228, 'pmin': 0.193, 'pmax': 0.25, 'sp':0.016}

MR={'KG': 150100000, 'ED1':37525000, 'LD1':28000000, 'UD1':49750000, 'ED2':112575000,'LD2':84000000, 
     'UD2':149250000, 'p':0.5956, 'pmin': 0.499, 'pmax': 0.721, 'sp':0.065}

PCV={'KG': 151836000, 'ED1':59216040, 'LD1':52260000, 'UD1':64740000, 'ED2':92619960,'LD2':81740000, 
     'UD2':101260000, 'p':4.022, 'pmin': 2.9, 'pmax': 7.0, 'sp':1.342}

Penta={'KG': 185588000, 'ED1':77946960, 'LD1':54600000, 'UD1':99120000, 'ED2':107641040,'LD2':75400000, 
     'UD2':136880000, 'p':0.9765, 'pmin': 0.65, 'pmax': 1.6, 'sp':0.329}

Rota={'KG': 60580000, 'ED1':21203000, 'LD1':13650000, 'UD1':28350000, 'ED2':39377000,'LD2':25350000, 
     'UD2':52650000, 'p':1.89, 'pmin': 0.85, 'pmax': 2.35, 'sp':0.717}

TT={'KG': 62256000, 'ED1':8093280, 'LD1':1596400, 'UD1':12090000, 'ED2':54162720,'LD2':10683600, 
     'UD2':80910000, 'p':0.051, 'pmin': 0.030, 'pmax': 0.066, 'sp':0.01}

YF={'KG': 55350000, 'ED1':0, 'LD1':0, 'UD1':0, 'ED2':55350000,'LD2':36000000, 
     'UD2':78000000, 'p':0.816, 'pmin': 0.66, 'pmax': 0.97, 'sp':0.113}



vac={ 'HPV':HPV, 'IPV':IPV, 'Measles':Measles, 'MR':MR, 'PCV':PCV, 'Penta':Penta, 'Rota':Rota, 'TT':TT, 'YF':YF}

## 6. Simulation execution 

**aa** : object that collects simulation output

#### Column headings of **aa**
- vac: vaccine
- g_rep: tuple for (g-value, replication)
- v2/v1: ratio between the profit-per-dose of a vaccine made and sold in AU Gavi countries and the profit-per-dose of the same vaccine if made and sold in non-AU Gavi countries. 
- v3/v1: ratio between the profit-per-dose of a vaccine made in non-AU Gavi countries and sold in AU Gavi countries and the the profit-per-dose of the same vaccine if made and sold in non-AU Gavi countries. 
- K1: non-AU vaccine production capacity
- K2: AU vaccine production capacity
- scenario: Profit scenario based on the relationship of the profit per dose for procurement scenarios (v1, v2, v3)
- area: profit scenario area based on the simulated vaccine profits and capacities
- xD: optimal profit-driven fulfillement for vaccines made and sold in non-AU countries (procurement interaction NN)
- xO: optimal profit-driven fulfillement for vaccines made and sold in AU countries (procurement interaction AA)
- xT: optimal profit-driven fulfillement for vaccines made in non-AU countries and sold in AU (procurement interaction NA)
- pfD:profit for procurement interaction NN (when there is AU production)
- pfO: profit for procurement interaction AA (when there is AU producton)
- pfT: profit for procurement interaction NA (when there is AU production)
- bpfD: benchmark profit for procurement interaction NN (when there is no AU production)
- bpfO: benchmark profit for procurement interaction AA (when there is no AU production)
- bpfT: benchmark profit for procurement interaction NA (when there is no AU production)
- pDtot: pfD + pfT
- bpfDtot: bpfD + bpfT
- g: proportion of the African vaccine demand that aims to be supplied with domestic production
- rep: replication 



In [20]:
aa=mainf(vac)

done with vac HPV
done with vac IPV
done with vac Measles
done with vac MR
done with vac PCV
done with vac Penta
done with vac Rota
done with vac TT
done with vac YF
Finally done!


## 7. Output collection

In [22]:
compression_opts = dict(method='zip',archive_name='aa.csv')  

aa.to_csv('aa.zip', index=False,compression=compression_opts)  

## Illustrative plotting code

In [21]:
import matplotlib.pyplot as plt  
myFig=plt.figure()

for i in vac:
    print("------------",i,"-----------------")
    aa[aa['vac']==i].boxplot(column=['pfD', 'bpfDtot'],by='scenario')
    plt.suptitle("FiguresA/"+i+"_pD_bpDtot")
    plt.plot()
    plt.savefig("FiguresA/"+i+"_pfD-bpfDtot.pdf",format="pdf")
    plt.close(plt.gcf())
        
    aa[aa['vac']==i].boxplot(column=['pfO', 'bpfO'],by='scenario')
    plt.suptitle(i+"_pO_bpO")
    plt.plot()
    plt.savefig("FiguresA/"+i+"_pfO-bpfO.pdf",format="pdf")
    plt.close(plt.gcf())
    

    aa[aa['vac']==i].boxplot(column=['pfT', 'bpfT'],by='scenario')
    plt.suptitle(i+"_pT_bpT")
    plt.plot()
    plt.savefig("FiguresA/"+i+"_pfT-bpfT.pdf",format="pdf")
    plt.close(plt.gcf())


------------ HPV -----------------
------------ IPV -----------------
------------ Measles -----------------
------------ MR -----------------
------------ PCV -----------------
------------ Penta -----------------
------------ Rota -----------------
------------ TT -----------------
------------ YF -----------------


<Figure size 432x288 with 0 Axes>